In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [3]:
from sklearn.preprocessing import LabelEncoder
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted','vehicle_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [4]:
y_train = train['target'] 

X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

(4701217, 18)
(4701217,)
(291241, 18)


In [5]:
def make_submission(pred):
    sample_submission = pd.read_csv('./submit.csv')
    sample_submission['target'] = pred
    sample_submission.to_csv("./submit.csv", index = False)
    print('save success')

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [7]:
from sklearn.ensemble import RandomForestRegressor
import warnings
rfr = RandomForestRegressor(random_state=10)
rfr.fit(X_train,y_train)
pred = rfr.predict(test)
# mae = mean_absolute_error(pred,y_test)
# print(mae)

In [8]:
make_submission(pred)

save success


In [23]:
from sklearn.ensemble import ExtraTreesRegressor
import warnings
et = ExtraTreesRegressor(random_state=10)
et.fit(X_train,y_train)
pred = et.predict(X_test)
mae = mean_absolute_error(pred,y_test)
print(mae)

3.5486579373710905


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    X_train,y_train,test_size=0.1,random_state=10
)

In [10]:
import catboost
cat = catboost.CatBoostRegressor(random_state=10,iterations=50,learning_rate=0.01)
cat.fit(X_train,y_train)
pred = cat.predict(X_test)
mae = mean_absolute_error(pred,y_test)
print(mae)

0:	learn: 15.8637076	total: 175ms	remaining: 8.55s
1:	learn: 15.7754466	total: 350ms	remaining: 8.41s
2:	learn: 15.6884601	total: 524ms	remaining: 8.21s
3:	learn: 15.6027340	total: 694ms	remaining: 7.99s
4:	learn: 15.5182535	total: 867ms	remaining: 7.8s
5:	learn: 15.4350054	total: 1.04s	remaining: 7.64s
6:	learn: 15.3530218	total: 1.22s	remaining: 7.47s
7:	learn: 15.2720923	total: 1.39s	remaining: 7.3s
8:	learn: 15.1929008	total: 1.56s	remaining: 7.13s
9:	learn: 15.1143257	total: 1.74s	remaining: 6.97s
10:	learn: 15.0369159	total: 1.92s	remaining: 6.82s
11:	learn: 14.9606539	total: 2.09s	remaining: 6.62s
12:	learn: 14.8855326	total: 2.27s	remaining: 6.46s
13:	learn: 14.8111105	total: 2.44s	remaining: 6.27s
14:	learn: 14.7384325	total: 2.61s	remaining: 6.1s
15:	learn: 14.6665289	total: 2.79s	remaining: 5.92s
16:	learn: 14.5957066	total: 2.96s	remaining: 5.74s
17:	learn: 14.5259320	total: 3.14s	remaining: 5.59s
18:	learn: 14.4572987	total: 3.33s	remaining: 5.43s
19:	learn: 14.3896232	tot

In [16]:
import xgboost
xgb = xgboost.XGBRegressor(random_state=10)
xgb.fit(X_train,y_train)
pred = xgb.predict(X_test)
mae = mean_absolute_error(pred,y_test)
print(mae)

4.033419129843013


In [22]:
pred = cat.predict(test)
make_submission(pred)

save success


- cat,rf,xgb

In [16]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor

r1 = RandomForestRegressor(random_state=10,criterion='absolute_error')
r2 = CatBoostRegressor(random_state=10)
r3 = ExtraTreesRegressor(random_state=10,criterion='absolute_error')
#r4 = XGBRFRegressor(random_state=10)
vote = VotingRegressor([('r1', r1), ('r2', r2), ('r3', r3)])
vote.fit(X_train,y_train)

Learning rate set to 0.153165
0:	learn: 14.6290142	total: 196ms	remaining: 3m 16s
1:	learn: 13.5853056	total: 381ms	remaining: 3m 10s
2:	learn: 12.7750529	total: 565ms	remaining: 3m 7s
3:	learn: 12.1478070	total: 741ms	remaining: 3m 4s
4:	learn: 11.6650937	total: 930ms	remaining: 3m 5s
5:	learn: 11.2180929	total: 1.11s	remaining: 3m 3s
6:	learn: 10.9082859	total: 1.28s	remaining: 3m 1s
7:	learn: 10.5983123	total: 1.48s	remaining: 3m 3s
8:	learn: 10.3462412	total: 1.66s	remaining: 3m 3s
9:	learn: 10.1725673	total: 1.83s	remaining: 3m 1s
10:	learn: 9.9759749	total: 2.01s	remaining: 3m 1s
11:	learn: 9.7933067	total: 2.18s	remaining: 2m 59s
12:	learn: 9.6748235	total: 2.36s	remaining: 2m 59s
13:	learn: 9.5473036	total: 2.54s	remaining: 2m 58s
14:	learn: 9.4381605	total: 2.71s	remaining: 2m 58s
15:	learn: 9.3429810	total: 2.9s	remaining: 2m 58s
16:	learn: 9.2817896	total: 3.08s	remaining: 2m 58s
17:	learn: 9.2145238	total: 3.25s	remaining: 2m 57s
18:	learn: 9.1375446	total: 3.43s	remaining:

158:	learn: 6.6479485	total: 28.5s	remaining: 2m 30s
159:	learn: 6.6416716	total: 28.7s	remaining: 2m 30s
160:	learn: 6.6348226	total: 28.9s	remaining: 2m 30s
161:	learn: 6.6297383	total: 29.1s	remaining: 2m 30s
162:	learn: 6.6255496	total: 29.3s	remaining: 2m 30s
163:	learn: 6.6197168	total: 29.4s	remaining: 2m 30s
164:	learn: 6.6143362	total: 29.6s	remaining: 2m 29s
165:	learn: 6.6059152	total: 29.8s	remaining: 2m 29s
166:	learn: 6.6017628	total: 30s	remaining: 2m 29s
167:	learn: 6.5982503	total: 30.1s	remaining: 2m 29s
168:	learn: 6.5911785	total: 30.3s	remaining: 2m 28s
169:	learn: 6.5815575	total: 30.5s	remaining: 2m 28s
170:	learn: 6.5753111	total: 30.6s	remaining: 2m 28s
171:	learn: 6.5723802	total: 30.8s	remaining: 2m 28s
172:	learn: 6.5676687	total: 31s	remaining: 2m 28s
173:	learn: 6.5624169	total: 31.2s	remaining: 2m 28s
174:	learn: 6.5578443	total: 31.4s	remaining: 2m 27s
175:	learn: 6.5527187	total: 31.6s	remaining: 2m 27s
176:	learn: 6.5460498	total: 31.7s	remaining: 2m 2

314:	learn: 5.9845852	total: 57.9s	remaining: 2m 5s
315:	learn: 5.9823566	total: 58s	remaining: 2m 5s
316:	learn: 5.9809359	total: 58.2s	remaining: 2m 5s
317:	learn: 5.9785920	total: 58.5s	remaining: 2m 5s
318:	learn: 5.9757951	total: 58.7s	remaining: 2m 5s
319:	learn: 5.9741853	total: 58.8s	remaining: 2m 5s
320:	learn: 5.9702526	total: 59s	remaining: 2m 4s
321:	learn: 5.9676963	total: 59.2s	remaining: 2m 4s
322:	learn: 5.9648726	total: 59.4s	remaining: 2m 4s
323:	learn: 5.9618963	total: 59.6s	remaining: 2m 4s
324:	learn: 5.9598846	total: 59.8s	remaining: 2m 4s
325:	learn: 5.9571470	total: 60s	remaining: 2m 4s
326:	learn: 5.9552683	total: 1m	remaining: 2m 3s
327:	learn: 5.9530336	total: 1m	remaining: 2m 3s
328:	learn: 5.9506658	total: 1m	remaining: 2m 3s
329:	learn: 5.9479004	total: 1m	remaining: 2m 3s
330:	learn: 5.9467764	total: 1m	remaining: 2m 3s
331:	learn: 5.9437351	total: 1m 1s	remaining: 2m 2s
332:	learn: 5.9404334	total: 1m 1s	remaining: 2m 2s
333:	learn: 5.9389780	total: 1m 1

469:	learn: 5.6930070	total: 1m 27s	remaining: 1m 38s
470:	learn: 5.6917607	total: 1m 27s	remaining: 1m 38s
471:	learn: 5.6879399	total: 1m 28s	remaining: 1m 38s
472:	learn: 5.6862895	total: 1m 28s	remaining: 1m 38s
473:	learn: 5.6852457	total: 1m 28s	remaining: 1m 38s
474:	learn: 5.6845886	total: 1m 28s	remaining: 1m 37s
475:	learn: 5.6836025	total: 1m 28s	remaining: 1m 37s
476:	learn: 5.6822029	total: 1m 28s	remaining: 1m 37s
477:	learn: 5.6813922	total: 1m 29s	remaining: 1m 37s
478:	learn: 5.6804469	total: 1m 29s	remaining: 1m 37s
479:	learn: 5.6792189	total: 1m 29s	remaining: 1m 36s
480:	learn: 5.6783412	total: 1m 29s	remaining: 1m 36s
481:	learn: 5.6761258	total: 1m 29s	remaining: 1m 36s
482:	learn: 5.6744056	total: 1m 30s	remaining: 1m 36s
483:	learn: 5.6734040	total: 1m 30s	remaining: 1m 36s
484:	learn: 5.6726850	total: 1m 30s	remaining: 1m 36s
485:	learn: 5.6711263	total: 1m 30s	remaining: 1m 35s
486:	learn: 5.6697634	total: 1m 30s	remaining: 1m 35s
487:	learn: 5.6686951	total:

621:	learn: 5.5436402	total: 1m 57s	remaining: 1m 11s
622:	learn: 5.5429252	total: 1m 57s	remaining: 1m 10s
623:	learn: 5.5424957	total: 1m 57s	remaining: 1m 10s
624:	learn: 5.5416799	total: 1m 57s	remaining: 1m 10s
625:	learn: 5.5408560	total: 1m 57s	remaining: 1m 10s
626:	learn: 5.5398554	total: 1m 57s	remaining: 1m 10s
627:	learn: 5.5389046	total: 1m 58s	remaining: 1m 10s
628:	learn: 5.5372442	total: 1m 58s	remaining: 1m 9s
629:	learn: 5.5358259	total: 1m 58s	remaining: 1m 9s
630:	learn: 5.5348721	total: 1m 58s	remaining: 1m 9s
631:	learn: 5.5341540	total: 1m 58s	remaining: 1m 9s
632:	learn: 5.5335203	total: 1m 59s	remaining: 1m 9s
633:	learn: 5.5327640	total: 1m 59s	remaining: 1m 8s
634:	learn: 5.5321726	total: 1m 59s	remaining: 1m 8s
635:	learn: 5.5314523	total: 1m 59s	remaining: 1m 8s
636:	learn: 5.5305225	total: 1m 59s	remaining: 1m 8s
637:	learn: 5.5297641	total: 2m	remaining: 1m 8s
638:	learn: 5.5288428	total: 2m	remaining: 1m 8s
639:	learn: 5.5282728	total: 2m	remaining: 1m 7

778:	learn: 5.4449041	total: 2m 27s	remaining: 41.9s
779:	learn: 5.4446242	total: 2m 27s	remaining: 41.7s
780:	learn: 5.4441576	total: 2m 27s	remaining: 41.5s
781:	learn: 5.4438342	total: 2m 28s	remaining: 41.3s
782:	learn: 5.4435180	total: 2m 28s	remaining: 41.1s
783:	learn: 5.4431275	total: 2m 28s	remaining: 40.9s
784:	learn: 5.4427346	total: 2m 28s	remaining: 40.7s
785:	learn: 5.4417806	total: 2m 28s	remaining: 40.6s
786:	learn: 5.4414466	total: 2m 29s	remaining: 40.4s
787:	learn: 5.4408256	total: 2m 29s	remaining: 40.2s
788:	learn: 5.4404254	total: 2m 29s	remaining: 40s
789:	learn: 5.4400224	total: 2m 29s	remaining: 39.8s
790:	learn: 5.4397264	total: 2m 29s	remaining: 39.6s
791:	learn: 5.4393061	total: 2m 30s	remaining: 39.4s
792:	learn: 5.4387678	total: 2m 30s	remaining: 39.2s
793:	learn: 5.4383166	total: 2m 30s	remaining: 39.1s
794:	learn: 5.4377217	total: 2m 30s	remaining: 38.9s
795:	learn: 5.4373497	total: 2m 30s	remaining: 38.7s
796:	learn: 5.4366682	total: 2m 31s	remaining: 3

935:	learn: 5.3796718	total: 2m 58s	remaining: 12.2s
936:	learn: 5.3794572	total: 2m 58s	remaining: 12s
937:	learn: 5.3792696	total: 2m 58s	remaining: 11.8s
938:	learn: 5.3790207	total: 2m 58s	remaining: 11.6s
939:	learn: 5.3785166	total: 2m 59s	remaining: 11.4s
940:	learn: 5.3783463	total: 2m 59s	remaining: 11.2s
941:	learn: 5.3781262	total: 2m 59s	remaining: 11s
942:	learn: 5.3777122	total: 2m 59s	remaining: 10.9s
943:	learn: 5.3774156	total: 2m 59s	remaining: 10.7s
944:	learn: 5.3772090	total: 3m	remaining: 10.5s
945:	learn: 5.3769458	total: 3m	remaining: 10.3s
946:	learn: 5.3765390	total: 3m	remaining: 10.1s
947:	learn: 5.3764288	total: 3m	remaining: 9.91s
948:	learn: 5.3761988	total: 3m	remaining: 9.72s
949:	learn: 5.3756173	total: 3m 1s	remaining: 9.53s
950:	learn: 5.3754086	total: 3m 1s	remaining: 9.34s
951:	learn: 5.3752588	total: 3m 1s	remaining: 9.15s
952:	learn: 5.3750839	total: 3m 1s	remaining: 8.96s
953:	learn: 5.3747044	total: 3m 1s	remaining: 8.77s
954:	learn: 5.3744871	

VotingRegressor(estimators=[('r1', RandomForestRegressor(random_state=10)),
                            ('r2',
                             <catboost.core.CatBoostRegressor object at 0x0000019F170396D0>),
                            ('r3', ExtraTreesRegressor(random_state=10)),
                            ('r4',
                             XGBRFRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eva...
                                            interaction_constraints=None,
                                            max_bin=None,
                                            max_cat_to_onehot=None,
                       

In [19]:
pred = vote.predict(X_test)

In [20]:
mae = mean_absolute_error(pred,y_test)
print(mae)

4.026297043170448
